In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## 1. Read the case, department, and source data into their own spark dataframes.

In [4]:
case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )


In [6]:
dept = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [3]:
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

## 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [8]:
# write to json
source.write.json("sources_json", mode="overwrite")

In [9]:
# write to csv
source.write.csv("sources_csv", mode="overwrite")

- On inspecting folder structure, I notice a success folder along with the files in each case.

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [13]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [12]:
# check dtypes for source dataframe
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

- source_id should be a numeric value. 

In [15]:
source = source.withColumn('source_id', col('source_id').cast('int'))

In [16]:
# check dtypes now
source.dtypes

[('source_id', 'int'), ('source_username', 'string')]

In [17]:
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [18]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

- case_opened_date,case_closed_date,SLA_due_date convert to date
- case_late, case_closed convert to boolean

In [21]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('SLA_due_date', to_timestamp('SLA_due_date', fmt))

In [19]:
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [22]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [52]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [56]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

- dept_subject_to_SLA convert to boolean.

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [71]:
(
    case.select(datediff(current_timestamp(), 'SLA_due_date')
    .alias('days_past_due'))
    .where(case.case_closed == False)
    .sort(col('days_past_due').desc())
    .show(5)
    
)

+-------------+
|days_past_due|
+-------------+
|         1662|
|         1662|
|         1662|
|         1661|
|         1659|
+-------------+
only showing top 5 rows



In [72]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?
(
    case.select(datediff(current_timestamp(), 'SLA_due_date')
    .alias('days_past_due'))
    .where(case.case_closed == False)
    .sort(col('days_past_due').desc())
    .show(5)
    
)

+-------------+
|days_past_due|
+-------------+
|         1662|
|         1662|
|         1662|
|         1661|
|         1659|
+-------------+
only showing top 5 rows



## 2. How many Stray Animal cases are there?

In [58]:
case.filter(case.service_request_type == 'Stray Animal').count()

26760

## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [42]:
(
    case.select('service_request_type')
    .where(case.dept_division == 'Field Operations')
    .where(case.service_request_type != 'Officer Standby').count()
)

113902

In [59]:
# alternative method
(
    case.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

113902

## 4. Convert the council_district column to a string column.

In [43]:
case = case.withColumn("council_district", col("council_district").cast("string"))


In [44]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('year', 'int')]

## 5. Extract the year from the case_closed_date column.

In [23]:
case = case.withColumn('year', year("case_closed_date"))

In [26]:
case.select(case.year).show(5)


+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 5 rows



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [62]:
(
    case.withColumn('num_hours_late', case.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show(3)
)


+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
+-------------------+-------------------+
only showing top 3 rows



## 7. Join the case data with the source and department data.

In [128]:
# join case and dept dfs
df = (
    case
    # do a left join so we keep everything in case df
    .join(dept, "dept_division", "left")
    # drop dept columns we don't need
    .drop(dept.dept_division)
    .drop(case.dept_division)
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [129]:
# join source df
df = df.join(source, on = df.source_id== source.source_id).drop(source.source_id)


In [130]:
df.show(3, vertical = True)


-RECORD 0--------------------------------------
 case_id                | 1014127378           
 case_opened_date       | 2018-01-01 08:24:00  
 case_closed_date       | 2018-01-02 07:37:00  
 SLA_due_date           | 2018-01-03 08:24:00  
 case_late              | false                
 num_days_late          | -1.03287037          
 case_closed            | true                 
 service_request_type   | Dead Animal - Cat    
 SLA_days               | 2.0                  
 case_status            | Closed               
 source_id              | 139344               
 request_address        | 7059  TIMBERCREEK... 
 council_district       | 6                    
 year                   | 2018                 
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | true                 
 source_username        | Isis Mendoza         
-RECORD 1--------------------------------------
 case_id                | 1014127380    

## 8. Are there any cases that do not have a request source?

In [131]:
case.where(expr("source_id is null")).show()

+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+----+
|case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|dept_division|service_request_type|SLA_days|case_status|source_id|request_address|council_district|year|
+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+----+
+-------+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+----+



## 9. What are the top 10 service request types in terms of number of requests?

In [132]:
(
    case_source_dept.select('service_request_type')
    .groupBy('service_request_type').count()
    .sort(desc('count')).show(10,truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



## 10. What are the top 10 service request types in terms of average days late?

In [133]:
(
    case.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|2      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



## 11. Does number of days late depend on department?


In [134]:
(df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.3      |382         |
|Solid Waste Management   |7.7      |16164       |
|Trans & Cap Improvements |9.5      |1947        |
|Animal Care Services     |23.2     |10419       |
|Parks and Recreation     |36.1     |241         |
|Code Enforcement Services|63.4     |2736        |
|Development Services     |69.4     |318         |
|Customer Service         |87.2     |995         |
+-------------------------+---------+------------+



In [135]:
df.groupby('dept_name').count().sort(desc('count')).show(truncate=False)


+-------------------------+------+
|dept_name                |count |
+-------------------------+------+
|Solid Waste Management   |132059|
|Animal Care Services     |47645 |
|Code Enforcement Services|36024 |
|Trans & Cap Improvements |29819 |
|Metro Health             |2357  |
|Customer Service         |1415  |
|Parks and Recreation     |1180  |
|Development Services     |481   |
|City Council             |20    |
+-------------------------+------+



## 12. How do number of days late depend on department and request type?


In [136]:
(
    df.filter("case_closed")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .where(col('days_late') > 0)
    .sort(desc("days_late"))
    .show(30, truncate=False)
)


+------------------------+----------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                    |days_late|n_cases|
+------------------------+----------------------------------------+---------+-------+
|DSD/Code Enforcement    |No Address Posted                       |132.0    |86     |
|DSD/Code Enforcement    |Vendors                                 |99.1     |42     |
|DSD/Code Enforcement    |Bandit Signs                            |93.0     |557    |
|DSD/Code Enforcement    |Curbstoning                             |91.4     |37     |
|Customer Service        |Complaint                               |59.8     |593    |
|DSD/Code Enforcement    |Street Light New Development            |55.7     |21     |
|Solid Waste             |Contracted Service Issues               |41.3     |6      |
|Animal Care Services    |Animal Permits Request                  |38.0     |928    |
|DSD/Code Enforcement    |Street Light Existing Res Su